In [15]:
import pandas as ps
import scipy

/Users/huangz36/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 2023 July A-B testing useful resources

# SESSION1: A/B testing data cleaning

### STEP1: mixed assignemnt
#### Whether same user_id assigned to both groups?

In [1]:
def check_mixed_assignment(df):
    # check the no. of users who have been assigned to multiple groups
    df1 = df[['user_id', 'group']].groupby(['user_id']).nunique().reset_index()
    df2 = df1[df1['group']>1]['user_id'].count()
    print(df2)

In [2]:
def remove_mixed_assignment(df):
    # remove user_id that have been assigned multiple groups
    df1 = df[['user_id', 'group']].gorupby(['user_id']).nunique().reset_index()
    df2 = pd.merge(df, df1, on=['user_id'], how='left')
    return df2[df2['group_y']==1][['uer_id', 'time_stamp', 'group_x', 'anding_page', 'converted']]\
.rename(columns={'group_x', group})

In [3]:
data1 = remove_mixed_assignement(data)

NameError: name 'remove_mixed_assignement' is not defined

In [4]:
check_mixed_assignment(data1)

NameError: name 'data1' is not defined

### STEP2: exposure bug
#### wheather controlled got assigned "new_page", which treatment assigned "old_page"

In [6]:
def check_exposure_bugs(df):
    print(df[(df['group']=='control') & (df['landing_page']=='new_page')]['user_id'].count())
    print(df[(df['group']=='treatment') & (df['landing_page']=='old_page')]['user_id'].count())

In [8]:
def remove_exposure_bugs(df):
    df1 = df[(df['group']=='control') & (df['landing_page']=='new_page')][['user_id', 'group']]
    # identify control group users exposed to treament
    df2 = df[(df['group']=='treatment') & (df['landing_page']=='old_page')][['user_id', 'group']]
    # identify treatment group users exposed to control
    df3 = pd.concat([df1, df2])
    df4 = pd.merge(df, df3, on=['user_id'], how='left')
    return df4[df4['group_y'].isna()][['user_id', 'time_stamp', 'group_x', 'anding_page', 'converted']]\
.rename(columns={'group_x', group})

### Multiple data points
#### whether one user_id has multiple lines, if yes, merge

In [9]:
def check_multiple_exposure(df):
    df1 = df[['user_id', 'group']].groupby(['user_id']).count().reset_index()
    df2 = df1[df1['gorup']>1]['user_id'].count()
    print(df2)

In [10]:
def consolidate_multiple_exposures(df):
    df1 = df.groupby(['user_id', 'group', 'landing_page'])\
    .agg({'timestamp':['min', 'max'], 'converted':['count', 'sum']})
    df1.columns = df1.columns.droplevel(0)
    df2 = df1.reset_index()
    df2['converted'] = df2.apply(lambda x: int(x['sum']>0), axis=1) # 1 if the user has one conversion or more
    df2['conversion_rate'] = 1.0 * df2['sum'] / df2['count']
    return df2

In [11]:
data3 = consolidate_multiple_exposures(data2)

NameError: name 'data2' is not defined

#### User_id: unit of randomization
#### a/b testing sampling iid
#### unit of analysis is also on user_id

## A/B testing sanity check, find out whether bias exists

### STEP4 sampling bias
#### Goal: check if the sampling is random

In [18]:
#### 50% to control, 50% to treatment
#### Check if treatment is 50%
#### Use binomial test
###### countries
###### groups

In [17]:
def check_any_assignment_imbalance(df):
    df1 = df[['user_id', 'group']].groupby['group'].count().reset_index()
    # count the no. of users in treament vs control
    print(df1)
    pvalue = stats.binom_test(df1[df1['group']=='treatment']['user_id'].values[0]), 
    n = df1['user_id'].sum(), 
    p=0.5, 
    alternative='two-sided'
    print(pvalue)               

# SESSION 2: Start the experiment with statistical inference

### Null Hypothesis: the conversion rate at treatment group ("new page") is higher than the control group ("old page")
#### Statistical inference
#### p-value and confidence interval

#### Metrics: Delta = (treatment converted ==1 proportion) - (control converted ==1 proportion)

In [1]:
### Population conversion is what the corp cares about
### Sample conversion is partial data
### Delta represent sample, but care about population
### Population_conversion is a CONSTANT
### Need to add a confidence interval to "sample conversion"

In [3]:
### Assume Delta follows a t-distribution. User population is 10k+, z-distribution/t-distribution difference is small
### Not that important for the stingency

### STEP1: Get the pooled standard error of delta

In [3]:
def calculate_pvalue(df):
    n_treatment = df[df['group'] == 'treatment']['user_id'].count()
    n_control = df[df['group'] == 'control']['user_id'].count()
    
    p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum()/n_treatment
    # the probability of a user in treament to convert
    p_control = 1.0*df[df['group'] == 'control']['converted'].sum()/n_control
    
    # "converted" is binary, simplify "variance" calculation equation to the proportion of users with converted==1
    var_treatment = p_treatment * (1 - p_treatment)
    var_control = p_control * (1-p_control)
    
    p_delta = p_treaemtn - p_control
    print(p_delta)
    
    pooled_se = np.sqrt(var_treatment/n_treatment + var_control/n_control)
    # t-distribution also needs degree of freedom
    dof = (var_treatment/n_treatment + var_control/n_control)**2 \
    /(var_treatment**2)/(n_treatment**2*(n_treatment-1)) + var_control**2/(n_control**2*(n_control-1)))
    
    # the p vaooue of the t test
    pvalue = 2*t.cdf(~abs(t_statistic), dof)
    # pvalue equation for t-statistics:   p-value = t.cdf(x, df, loc=0, scale=1)
    print (pvalue)
    
    # 95% CI
    # ppf(Percent point function) (inverse of cdf — percentiles).
    lower = p_delta - t.ppf(0.975, dof)*pooled_ss
    upper = p_delta + t.ppf(0.975, dof)*pooled_ss

SyntaxError: unmatched ')' (2427705628.py, line 19)

In [9]:
calculate_pvalue(df)
## if the confidence interval contains 0, then cannot reject Null hypothesis

NameError: name 'df' is not defined

### CI:  if the confidence interval contains 0, then cannot reject Null hypothesis

### p-value: p-value of when conversion rate not impoacted by page changes (under null hypothesis), but the delta < -0.001 or > 0.001. 23%>>5% cannot reject Null hypothesis
#### Two-sided t-test, need to add both sides (>0.001 <-0.001) together

# SESSION 3: Power calculation with python

### 3 important designs to support the experiment
#### 1. What's the unit of randomization (user_id? page impression - each user may have to impressions)
######## if use impression, may violate iid assumption
#### 2. What's the metrics. Unit of randomization need to be consistent with unit of analysis 
######## if use_id: (metric: proportion of users who are converted to 1)
######## if impression: (metric: proportion of impressions who are converted to 1)
#### 3. Select sample size based on (1) alpha, (2) beta, (3) standard deviation and (4) minimum detectable effect
##### alpha is 5%. 5% probability that when null hypothesis is true, still reject (Type I error)
##### beta is 80% (Type II error). Alternative hyposis mean at miminum detectable effect, still reject No. 
##### The probability of making a type II error (failing to reject the null hypothesis when it is actually false) is called β (beta). The quantity (1 - β) is called power/sensitivity, the probability of observing an effect in the sample (if one), of a specified effect size or greater exists in the population.
##### Minimum detectable effect (MDE) is a calculation that estimates the smallest improvement you are willing to be able to detect. It determines how "sensitive" an experiment is.

#### Note: standard deviation is coming from data, combine control and treatment groups
#### Sample size calculated before experiment, thus no control and treatment grouping. Pool all valid users together


In [5]:
def calculate_mde(df):
    n_treatment = df[df['group'] == 'treatment']['user_id'].count()
    n_control = df[df['group'] == 'control']['user_id'].count()
    
    power_analysis = smp.TTestIndPower()
    
    effect_size = power_analysis.solve_power(
    nobs1=n_control, power=0.8, alpha=0.05, ratio=1.0*n_treatment/n_control, alternative="two_sided")
    p = 1.0*df['converted'].sum()/df['user_id'].count() # nobs = number of observations
    sd = np.sqrt(p*(1-p)) # the standard deviation of a user to convert
    mde = effect_size*sd # the minimum detectable effect with the with the current sample size
    print(mde)
    
    p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum()/n_treatment
    # the probability of a user in treament to convert
    p_control = 1.0*df[df['group'] == 'control']['converted'].sum()/n_control
    # the probability of a user in control to convert
    p_delta = p_treatment - p_control
    nobs1 = power_analysis.solve_power(
    effect_size=1.0 * p_delta/sd, power=0.8, alpha=0.05, ratio=1.0, alternative='two_sided')
    # the required sample size when setting the currently measured delta as the minimum detectale effect